In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [2]:
# dictionaries downloading
df = pd.read_excel("appl_register.xlsx", sheet_name="data")
managers = pd.read_excel("dictionary.xlsx", sheet_name="managers")
units = pd.read_excel("dictionary.xlsx", sheet_name="units")
unload_addresses = pd.read_excel("dictionary.xlsx", sheet_name="unload_addresses")

In [3]:
df

,Менеджер,Вид доставки,Дата,Товар,Примечание к Товару,Кол-во,Ед.изм.,Машина/Водитель,Продавец,Откуда,Покупатель,Грузополучатель,Юр. адрес грузополучателя,Адрес пункта разгрузки,Контакт гп,Время приемки,Примечание Иное,Текст заявки
0,Игорь Хабаров,доставка,26.04.2024,"ЦЕМ I 42,5н",Сертификаты!,35.0,т,"У162ХК750 MAN, Х240ВА797 КАМАЗ, А215УХ750 МAN","ООО ""Спарта""",Солнечногорска,"ООО ""Бетон Строй»","ООО ""Бетон Строй»","249850, Калужская обл., Дзержинский р-н, д. Об...","город Солнечногорск, ул. Промышленная 1б","+79232208000, +74959242828",с 9.00 до 20.00 Круглосуточно,оплата 250,NaN
1,Игорь Хабаров,доставка,26.04.2024,ЦЕМ I 42.5Н БЦК,NaN,35.0,т,NaN,"ООО ""Спарта""",псо 13,"ООО ""Бетонная индустрия»","ООО ""Бетонная индустрия»",\nОдинцово,Одинцово. Кобяковская. Краснознаменск.,NaN,NaN,NaN,NaN
2,Юрий,доставка,26.04.2024,Щебень гранитный 5-20(ЛСР),Обязательно актуальный паспорт!,120.0,т,"О327ВН790 МАN, О039ОУ790 МАN, Х194ВА797 КАМАЗ,...",ООО Спарта,NaN,ООО НВЛ ГРУП,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Игорь Хабаров,доставка,25.04.2024,ЦЕМ I 42.5Н БЦК,NaN,35.0,т,NaN,"ООО ""Спарта""",псо 13,"ООО ""Бетонная индустрия»","ООО ""Бетонная индустрия»",\nОдинцово,Одинцово. Кобяковская. Краснознаменск.,8 (903) 007-00-86,NaN,NaN,NaN
4,Алексей Мельхер,доставка,25.04.2024,"ЦЕМ I 42,5Н Костюковичи",NaN,40.0,т,В894ВХ790 Мурадов,"ООО ""СЗТК""",Люберецы,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,Игорь Хабаров,доставка,29.04.2024,": ЦЕМ I 42,5 Н Кричев/Костюковичи",NaN,140.0,т,NaN,ООО Спарта,"КПП, Солнечногорск, Люберцы",ООО «Сибирский элемент Рента - К»,ООО «Сибирский элемент Рента - К»,"249850, Калужская обл., Дзержинский р-н, д. Об...","город Солнечногорск, ул. Промышленная 1б",8 (965) 709-90-77,круглосуточно,Оплата мильён,Заявка на доставку 1. Дата отгрузки: 29.04...
64,Игорь ХабаровИгорь Хабаров,доставка,29.04.2024,": ЦЕМ I 42,5 Н Кричев/Костюковичи",NaN,140.0,т,NaN,ООО Спарта,"КПП, Солнечногорск, Люберцы",ООО «Сибирский элемент Рента - К»,ООО «Сибирский элемент Рента - К»,"249850, Калужская обл., Дзержинский р-н, д. Об...","город Солнечногорск, ул. Промышленная 1б",8 (965) 709-90-77,круглосуточно,Оплата мильён,Заявка на доставку 1. Дата отгрузки: 29.04...
65,Игорь ХабаровИгорь Хабаров,доставка,29.04.2024,": ЦЕМ I 42,5 Н Кричев/Костюковичи",NaN,10.0,т,NaN,ООО Спарта,"КПП, Солнечногорск, Люберцы",ООО «Сибирский элемент Рента - К»,ООО «Сибирский элемент Рента - К»,"249850, Калужская обл., Дзержинский р-н, д. Об...","город Солнечногорск, ул. Промышленная 1б",8 (965) 709-90-77,круглосуточно,Оплата мильён,Заявка на доставку 1. Дата отгрузки: 29.04...
66,Игорь Хабаров,доставка,29.04.2024,": ЦЕМ I 42,5 Н Кричев/Костюковичи",NaN,10.0,т,NaN,ООО Спарта,"КПП, Солнечногорск, Люберцы",ООО «Сибирский элемент Рента - К»,ООО «Сибирский элемент Рента - К»,"249850, Калужская обл., Дзержинский р-н, д. Об...","город Солнечногорск, ул. Промышленная 1б",8 (965) 709-90-77,круглосуточно,Оплата мильён,Заявка на доставку 1. Дата отгрузки: 29.04...


In [6]:
appl = str(input('Введите заявку'))

Введите заявку\nЗаявка на доставку  \n1. Дата отгрузки:  29.04\n2. Марка цемента: ЦЕМ I 42,5 Н Кричев/Костюковичи\n3. Кол-во 10 тонн\n4. Продажа от клиента: ООО Спарта    \n5. Завод отгрузки: КПП, Солнечногорск, Люберцы \n6. Покупатель груза: ООО «Сибирский элемент Рента - К»    \n7. Грузополучателя (при оформлении ТТН): ООО «Сибирский элемент Рента - К»    \n8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    \n9. Пункт разгрузки (факт): Калужская обл., Дзержинский р-н, д. Обухово 1Б    \n10. Плательщик транспортировки (при оформлении ТТН): \n Оплата мильён   \nКонтактный телефон 8 (965) 709-90-77.\nПриёмка круглосуточно\n


In [7]:
appl

'\\nЗаявка на доставку  \\n1. Дата отгрузки:  29.04\\n2. Марка цемента: ЦЕМ I 42,5 Н Кричев/Костюковичи\\n3. Кол-во 10 тонн\\n4. Продажа от клиента: ООО Спарта    \\n5. Завод отгрузки: КПП, Солнечногорск, Люберцы \\n6. Покупатель груза: ООО «Сибирский элемент Рента - К»    \\n7. Грузополучателя (при оформлении ТТН): ООО «Сибирский элемент Рента - К»    \\n8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    \\n9. Пункт разгрузки (факт): Калужская обл., Дзержинский р-н, д. Обухово 1Б    \\n10. Плательщик транспортировки (при оформлении ТТН): \\n Оплата мильён   \\nКонтактный телефон 8 (965) 709-90-77.\\nПриёмка круглосуточно\\n'

In [5]:
# processing of application data
# extracting dates from application
def find_dates(text):
    """
    Function for dates extracting 
    """
    date_pattern = r'\d{2}\.(?:0[1-9]|1[0-2])(?:\.\d{2}(?:\d{2})?)?'
    dates = re.findall(date_pattern, appl)
    
    return dates

In [ ]:
found_dates = find_dates(appl)

In [6]:
dates

['28.04']

In [7]:
def convert_to_full_year(date_str):
    """
    Function for formatting the date into the specified format
    """
    current_year = datetime.now().year
    parts = date_str.split('.')

    try:
        if len(parts) == 2:
            date_str += f".{current_year}"

        datetime_obj = datetime.strptime(date_str, '%d.%m.%Y')
    except ValueError:
        try:
            datetime_obj = datetime.strptime(date_str, '%d.%m.%y')
        except ValueError:

            raise ValueError("Неподдерживаемый формат даты")

    return datetime_obj.strftime('%d.%m.%Y')

In [8]:
# assigning a variable with a date
convert_date = convert_to_full_year(found_dates[0])

In [9]:
convert_date

'28.04.2024'

In [10]:
def find_manager(text, managers_df):
    """
    Function for getting the manager's name by login from text
    """
    for index, row in managers_df.iterrows():
        login = row['login']
        manager = row['manager']

        if login.lower().replace(' ', '') in text.lower().replace(' ', ''):
            return manager
    
    # Если ни один менеджер не найден, вернуть np.NAN
    return np.NAN

In [11]:
managers

,login,manager
0,ИГО,Игорь Хабаров
1,Юра Менеджер,Юрий
2,Алексей Мельхер,Алексей Мельхер


In [12]:
# assigning a variable with the manager's name
manager_found = find_manager(appl, managers)

In [13]:
manager_found

nan

In [78]:
def delivery_type(text):
    """
    Function for determining the type of delivery
    """
    if "самовывоз" in text.lower():
        return "самовывоз"
    elif "автономка" in text.lower():
        return "автономка доставка"
    else:
        return "доставка"

In [79]:
# assigning a variable with the delivery's type
delivery_found = delivery_type(appl)

In [80]:
delivery_found

'автономка доставка'

# HERE

In [15]:
def find_product(text):
    """
    Function for determining the type of product
    """
    marca_match = re.search(r'(?i)Марка(:)?\s*(цемента)?\s*(.*?)\\n', text)
    marca_value = marca_match.group(3) if marca_match else np.NAN

    return marca_value

In [16]:
# assigning a variable with the product's type
product_found = find_product(appl)

In [17]:
product_found

'ЦЕМ I 42.5Н Костюковичи '

# HERE

In [18]:
def find_product_notice(text):
    """
    Function for determining the notice for the product
    """
  
    intermediate_match = re.search(r'(?i)Марка.*?\\n(.*?)\\n\d', text, re.DOTALL)
    intermediate_value = intermediate_match.group(1) if intermediate_match else np.NAN
    
    if re.match(r'^[^0-9.].*', intermediate_value):
        return intermediate_value
    else:
        np.NAN

In [19]:
appl

'\\nАлексей Мельхер:\\nЗаявка на доставку \\n1.Дата отгрузки 28.04\\n2. Марка: ЦЕМ I 42.5Н Костюковичи \\n3. Количество: 70 тонн     \\n4. Продажа от ООО "СЗТК" \\n5. С Солнечногорска \\n6. Покупатель ООО «М Бетон» \\n7. Грузополучатель ООО «М-Бетон» \\n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \\n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\\n10. Контакты для связи 8-925-758-01-54\\n'

In [20]:
# assigning a variable with the product's notice
product_notice_found = find_product_notice(appl)

In [21]:
product_notice_found

In [22]:
def find_quantity(text):
    """
    Function for determining the product's quantity
    """
    quantity_match = re.search(r'(?i)кол(\s*-\s*|ичест)?во\s*(?:\D*)(:)?\s*(\d+)', text)
    quantity = int(quantity_match.group(3)) if quantity_match else np.NAN

    return quantity

In [23]:
# assigning a variable with the product's quantity
quantity_found = find_quantity(appl)

In [24]:
quantity_found

70

# HERE

In [25]:
appl

'\\nАлексей Мельхер:\\nЗаявка на доставку \\n1.Дата отгрузки 28.04\\n2. Марка: ЦЕМ I 42.5Н Костюковичи \\n3. Количество: 70 тонн     \\n4. Продажа от ООО "СЗТК" \\n5. С Солнечногорска \\n6. Покупатель ООО «М Бетон» \\n7. Грузополучатель ООО «М-Бетон» \\n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \\n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\\n10. Контакты для связи 8-925-758-01-54\\n'

In [26]:
def find_unit_note(text):
    """
    Function for determining the quantity's unit
    """
    unit_match = re.search(r'(?i)кол(-|ичест)?во\s*(?:\D*)(:)?\s*(\d+)\s*(\D+)\s*\\n', text)
    unit = unit_match.group(4).strip() if unit_match else np.NAN

    return unit

In [27]:
# assigning a variable with the quantity's unit
unit_note_found = find_unit_note(appl)

In [28]:
unit_note_found

'тонн'

In [29]:
# finding data in a dictionary
unit_found = units.loc[units[units['data'] == unit_note_found].index[0], 'unit']

In [30]:
unit_found

'т'

In [31]:
def find_car(text):
    """
    Function for determining the car's numbers
    """
    car_match = re.findall(r'(?i)[А-Я]{1}\d{3}[А-Я]{2}\d{3}\s*\w*', text)
    cars = car_match if car_match else np.NAN

    return cars

In [32]:
# assigning a variable with car numbers and writing them without list brackets
cars_found = ', '.join(find_car(appl)) if find_car(appl) is not np.NAN else np.NAN

In [33]:
cars_found

nan

# HERE

In [34]:
def find_organization(text):
    """
    Function for determining our organization's name
    """
    organization_match = re.search(r'(?i)(продажа\s+от:|(продажа)?\s*от\s+(клиента)?\s*(:)?)\s*(.+?)\\n', text)
    organization = organization_match.group(5) if organization_match else np.NAN

    return organization

In [35]:
# assigning a variable with our organization's name
organization_found = find_organization(appl)

In [36]:
organization_found

'ООО "СЗТК" '

# HERE

In [37]:
def find_transshipment(text):
    """
    Function for determining the transshipment's point
    """
    transshipment_match = \
        re.search(r'(?i)\d+\.\s*(С\s+(перевалки)?\s*|Завод\s*(отгрузки)?\s*(:)?|Перевалка\s*(:)?)\s*(.+?)\\n',
                  text)
    transshipment = transshipment_match.group(6) if transshipment_match else np.NAN

    return transshipment

In [38]:
# assigning a variable with the transshipment's point
transshipment_found = find_transshipment(appl)

In [39]:
transshipment_found

'Солнечногорска '

# HERE

In [40]:
def find_purchaser(text):
    """
    Function for determining the purchaser's name
    """
    purchaser_match = \
        re.search(r'(?i)\d+\.\s*(Покупатель\s*(груза)?\s*(:)?)\s*(.+?)\\n',
                  text)
    purchaser = purchaser_match.group(4) if purchaser_match else np.NAN

    return purchaser

In [41]:
# assigning a variable with the purchaser's name
purchaser_found = find_purchaser(appl)

In [42]:
purchaser_found

'ООО «М Бетон» '

# HERE

In [43]:
def find_consignee(text):
    """
    Function for determining the consignee's name
    """
    consignee_match = \
        consignee_match = re.search(
        r'(?i)\d+\.\s*(?:Грузополучатель\s*(?::)?|Грузополучатель\s*\(при\s*оформлении\s*ттн\)\s*(?::)?)\s*(.+?)\\n',
        text)
    consignee = consignee_match.group(1).split(':')[-1].strip() if consignee_match else np.NAN

    return consignee

In [44]:
# assigning a variable with the consignee's name
consignee_found = find_consignee(appl)

In [45]:
consignee_found

'ООО «М-Бетон»'

# HERE 

In [46]:
appl

'\\nАлексей Мельхер:\\nЗаявка на доставку \\n1.Дата отгрузки 28.04\\n2. Марка: ЦЕМ I 42.5Н Костюковичи \\n3. Количество: 70 тонн     \\n4. Продажа от ООО "СЗТК" \\n5. С Солнечногорска \\n6. Покупатель ООО «М Бетон» \\n7. Грузополучатель ООО «М-Бетон» \\n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \\n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\\n10. Контакты для связи 8-925-758-01-54\\n'

In [47]:
# r'(?i)\d+\.\s*(Юр\s*\.)?\s*(Адрес\s*грузополучателя)\s*(\(юрид\))?\s*(:)?\s*(.+?)\\n'
# r'(?i)\d+\.\s*(юр\s*\.\s*адрес\s*грузополучателя|адрес\s*грузополучателя\s*\(юрид\))\s*(?::)?\s*(.+?)\\n'
def find_consignee_leg_addr(text):
    """
    Function for determining the legal consignee's address
    """
    find_consignee_leg_addr_match = \
        re.search(r'(?i)\d+\.\s*(юр\w*\s*(?:\.)?\s*адрес\s*грузополучателя|адрес\s*грузополучателя\s*\(юр\w*\s*(?:\.)?\))\s*(?::)?\s*(.+?)\\n',
                  text)
    find_consignee_leg_addr = find_consignee_leg_addr_match.group(2) if find_consignee_leg_addr_match else np.NAN

    return find_consignee_leg_addr

In [48]:
# assigning a variable with legal consignee's address
consignee_leg_addr_found = find_consignee_leg_addr(appl)

In [49]:
consignee_leg_addr_found

'109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 '

In [50]:
def find_phones(text):
    """
    Function for determining the phone numbers
    """
    phone_numbers = re.findall(r'\+?\d{1,3}[\s-]?\(?\d{3}\)?[\s-]?\d{2,3}[\s-]?\d{2}[\s-]?\d{2}', text)
    phone_numbers = phone_numbers if phone_numbers else np.NAN

    return phone_numbers

In [51]:
appl

'\\nАлексей Мельхер:\\nЗаявка на доставку \\n1.Дата отгрузки 28.04\\n2. Марка: ЦЕМ I 42.5Н Костюковичи \\n3. Количество: 70 тонн     \\n4. Продажа от ООО "СЗТК" \\n5. С Солнечногорска \\n6. Покупатель ООО «М Бетон» \\n7. Грузополучатель ООО «М-Бетон» \\n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \\n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\\n10. Контакты для связи 8-925-758-01-54\\n'

In [52]:
find_phones(appl)

['8-925-758-01-54']

In [53]:
# assigning a variable with phone numbers and writing them without list brackets
phones_found = ', '.join(find_phones(appl)) if find_phones(appl) is not np.NAN else np.NAN

In [54]:
phones_found

'8-925-758-01-54'

In [55]:
def find_unl_addr(text, unload_addresses_df):
    """
    Function for determining unload addresses
    """
    for index, row in unload_addresses_df.iterrows():
        pattern1 = row['pattern1']
        pattern2 = row['pattern2']
        address = row['address']

        if pattern1.lower().replace(' ', '') and pattern2.lower().replace(' ', '') \
                in text.lower().replace(' ', ''):
            return address
    return np.NAN

In [56]:
# assigning a variable with unload addresses
unl_addr_found = find_unl_addr(appl, unload_addresses)

In [57]:
unl_addr_found

'город Солнечногорск, ул. Промышленная 1б '

In [58]:
appl

'\\nАлексей Мельхер:\\nЗаявка на доставку \\n1.Дата отгрузки 28.04\\n2. Марка: ЦЕМ I 42.5Н Костюковичи \\n3. Количество: 70 тонн     \\n4. Продажа от ООО "СЗТК" \\n5. С Солнечногорска \\n6. Покупатель ООО «М Бетон» \\n7. Грузополучатель ООО «М-Бетон» \\n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \\n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\\n10. Контакты для связи 8-925-758-01-54\\n'

In [59]:
def find_time(text):
    """
    Function for determining the unloading time
    """
    time_match = re.search(r'(?i)(время)?\s*при(ё|е)мк(и|а)(?::)?\s*(.*?)\s*\\n', text)
    time = time_match.group(4) if time_match else np.NAN

    return time

In [60]:
# assigning a variable with unloading time
accept_time_found = find_time(appl)

In [61]:
accept_time_found

nan

# HERE

In [68]:
appl

'\\nАлексей Мельхер:\\nЗаявка на доставку \\n1.Дата отгрузки 28.04\\n2. Марка: ЦЕМ I 42.5Н Костюковичи \\n3. Количество: 70 тонн     \\n4. Продажа от ООО "СЗТК" \\n5. С Солнечногорска \\n6. Покупатель ООО «М Бетон» \\n7. Грузополучатель ООО «М-Бетон» \\n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \\n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\\n10. Контакты для связи 8-925-758-01-54\\n'

In [69]:
def find_note(text):
    """
    Function for determining the note
    """
    if 'оплата' in text:
        note_match = re.search(r'(?i)(оплата)\s*(?::)?\s*(.*?)\\n', text)
        note = '{} {}'.format(note_match.group(1).strip(), note_match.group(2).strip() if note_match else np.NAN)
        return note
    else:
        return np.NAN

In [71]:
# assigning a variable with the note
note_found = find_note(appl)

In [72]:
note_found

nan

In [73]:
# creating new df with a string
new_row_df = pd.DataFrame({'Менеджер': [manager_found],
                           'Вид доставки': [delivery_found],
                           'Дата': [convert_date],
                           'Товар': [product_found],
                           'Примечание к Товару': [product_notice_found],
                           'Кол-во': [quantity_found],
                           'Ед.изм.': [unit_found],
                           'Машина/Водитель': [cars_found],
                           'Продавец': [organization_found],
                           'Откуда': [transshipment_found],
                           'Покупатель': [purchaser_found],
                           'Грузополучатель': [consignee_found],
                           'Адрес грузополучателя (юрид)': [consignee_leg_addr_found],
                           'Адрес пункта разгрузки': [unl_addr_found],
                           'Контакт гп': [phones_found],
                           'Время приемки': [accept_time_found],
                           'Примечание Иное': [note_found],
                          })

In [74]:
# concatenating dfs
df = pd.concat([df, new_row_df], ignore_index=True)

In [5]:
# record updated df
with pd.ExcelWriter("appl_register.xlsx") as writer:
    df.to_excel(writer, sheet_name="data", index=False)

In [4]:
df.drop(df.index, inplace=True)

In [6]:
"""
ИГО:
1. Дата отгрузки
26.04.2024
2.Марка цемента ЦЕМ I 42.5Н БЦК 
3. Количество 35 тонн 
4. Продажа от ООО "Спарта"
5. С псо 13
6. Покупатель ООО "Бетонная индустрия»
7. Грузополучатель ООО "Бетонная индустрия»
8. Адрес грузополучателя 
Одинцово 
Кобяковская. Краснознаменск.
"""

'\nИГО:\n1. Дата отгрузки\n26.04.2024\n2.Марка цемента ЦЕМ I 42.5Н БЦК \n3. Количество 35 тонн \n4. Продажа от ООО "Спарта"\n5. С псо 13\n6. Покупатель ООО "Бетонная индустрия»\n7. Грузополучатель ООО "Бетонная индустрия»\n8. Адрес грузополучателя \nОдинцово \nКобяковская. Краснознаменск.\n'

In [26]:
"""
Юра Менеджер:
1. Дата отгрузки:
26.04.2024
2. От ООО Спарта
3. Марка: Щебень гранитный 5-20(ЛСР)
Обязательно актуальный паспорт!
4. Покупатель ООО НВЛ ГРУП
5. Бетас 
6.  Количество 120т
7. Машина: 
О327ВН790 МАN
О039ОУ790 МАN,
Х194ВА797 КАМАЗ
В683СН790 МАN
М991ХС750 МАN
О030ХЕ123 МАN
В551ВО790 МАN
С289УТ750 МАN
В247ХО750 МАN 
У162ХК750 MAN
Х240ВА797 КАМАЗ
 А215УХ750 МAN
"""

'\nЮра Менеджер:\n1. Дата отгрузки:\n26.04.2024\n2. От ООО Спарта\n3. Марка: Щебень гранитный 5-20(ЛСР)\nОбязательно актуальный паспорт!\n4. Покупатель ООО НВЛ ГРУП\n5. Бетас \n6.  Количество 120т\n7. Машина: \nО327ВН790 МАN\nО039ОУ790 МАN,\nХ194ВА797 КАМАЗ\nВ683СН790 МАN\nМ991ХС750 МАN\nО030ХЕ123 МАN\nВ551ВО790 МАN\nС289УТ750 МАN\nВ247ХО750 МАN \nУ162ХК750 MAN\nХ240ВА797 КАМАЗ\n А215УХ750 МAN\n'

In [27]:
"""
ИГО:
1. Дата отгрузки
25.04.2024
2.Марка цемента ЦЕМ I 42.5Н БЦК 
3. Количество 35 тонн 
4. Продажа от ООО "Спарта"
5. С псо 13
6. Покупатель ООО "Бетонная индустрия»
7. Грузополучатель ООО "Бетонная индустрия»
8. Адрес грузополучателя 
Одинцово 
Кобяковская. Краснознаменск. 
8 (903) 007-00-86
"""

'\nИГО:\n1. Дата отгрузки\n25.04.2024\n2.Марка цемента ЦЕМ I 42.5Н БЦК \n3. Количество 35 тонн \n4. Продажа от ООО "Спарта"\n5. С псо 13\n6. Покупатель ООО "Бетонная индустрия»\n7. Грузополучатель ООО "Бетонная индустрия»\n8. Адрес грузополучателя \nОдинцово \nКобяковская. Краснознаменск. \n8 (903) 007-00-86\n'

In [28]:
"""
Алексей Мельхер:
Заявка/наша доставка
1. 25.04
2. От ООО "СЗТК"
3. МАРКА  ЦЕМ I 42,5Н Костюковичи 
4. ООО «ПД Технологии» объект *Пушкино* 
5. С перевалки Люберецы
6. Количество 40 тонн
7. Машина В894ВХ790 Мурадов"""

'\nАлексей Мельхер:\nЗаявка/наша доставка\n1. 25.04\n2. От ООО "СЗТК"\n3. МАРКА  ЦЕМ I 42,5Н Костюковичи \n4. ООО «ПД Технологии» объект *Пушкино* \n5. С перевалки Люберецы\n6. Количество 40 тонн\n7. Машина В894ВХ790 Мурадов'

In [29]:
"""
Юра Менеджер:
Заявка/ самовывоз
1. Дата отгрузки: 26.04.24
2. Марка цемента: ЦЕМ I 42.5Н БЦЗ Костюковичи 
3. Кол-во машин/ тонн: 100т
4. Продажа от клиента: ОО0 Спарта 
5. Завод отгрузки: СЗТК
6. Покупатель груза:ООО "Форма ЖБИ"
7. Грузополучатель (при оформлении ттн): ООО "Форма ЖбИ"
8. М192МС90"""

'\nЮра Менеджер:\nЗаявка/ самовывоз\n1. Дата отгрузки: 26.04.24\n2. Марка цемента: ЦЕМ I 42.5Н БЦЗ Костюковичи \n3. Кол-во машин/ тонн: 100т\n4. Продажа от клиента: ОО0 Спарта \n5. Завод отгрузки: СЗТК\n6. Покупатель груза:ООО "Форма ЖБИ"\n7. Грузополучатель (при оформлении ттн): ООО "Форма ЖбИ"\n8. М192МС90'

In [26]:
"""
Алексей Мельхер:
Заявка/наша доставка
1. 25.04
2. От ООО "СЗТК"
3. МАРКА  ЦЕМ I 42,5Н Костюковичи 
4. ООО «ПД Технологии» объект *Пушкино* 
5. С перевалки Люберецы
6. Количество 40 тонн
7. К702ОС797 Куряев"""

'\nАлексей Мельхер:\nЗаявка/наша доставка\n1. 25.04\n2. От ООО "СЗТК"\n3. МАРКА  ЦЕМ I 42,5Н Костюковичи \n4. ООО «ПД Технологии» объект *Пушкино* \n5. С перевалки Люберецы\n6. Количество 40 тонн\n7. К702ОС797 Куряев'

In [11]:
"""
ИГО:
Заявка на доставку
1. Дата отгрузки на 26.04
2. Марка цемента ЦЕМ I 42,5н 
3. Количество 35 тонн 
4. Продажа от ООО "Спарта"
5. С Солнечногорска  
6. Покупатель ООО "Бетон Строй»
7. Грузополучатель ООО "Бетон Строй»
8. Адрес город Солнечногорск, ул. Промышленная 1б 
+79232208000
+74959242828
Дебет оплата 250
"""

'\nИГО:\nЗаявка на доставку\n1. Дата отгрузки на 26.04\n2. Марка цемента ЦЕМ I 42,5н \n3. Количество 35 тонн \n4. Продажа от ООО "Спарта"\n5. С Солнечногорска  \n6. Покупатель ООО "Бетон Строй»\n7. Грузополучатель ООО "Бетон Строй»\n8. Адрес город Солнечногорск, ул. Промышленная 1б \n+79232208000\n+74959242828\nДебет оплата 250\n'

In [12]:
"""
Юра Менеджер:
Заявка на доставку
1. Дата отгрузки: 27.04.24 
2. Марка: ПЕСКОГРУНТ
3. Количество: 200 кубов 
4. Продажа от: ООО "Спарта"
5. Перевалка: 🤷‍♂️
6. Покупатель: АО "МСУ-1"
7. Грузополучатель: АО "МСУ-1"
8. Время приёмки: с 9.00 до 20.00
9. Контакт диспетчера: +7 968 558-83-13 
10. Точка выгрузки: Бескудниковский бульвар, 52к1
Россия, Москва
"""

'\nЮра Менеджер:\nЗаявка на доставку\n1. Дата отгрузки: 27.04.24 \n2. Марка: ПЕСКОГРУНТ\n3. Количество: 200 кубов \n4. Продажа от: ООО "Спарта"\n5. Перевалка: 🤷\u200d♂️\n6. Покупатель: АО "МСУ-1"\n7. Грузополучатель: АО "МСУ-1"\n8. Время приёмки: с 9.00 до 20.00\n9. Контакт диспетчера: +7 968 558-83-13 \n10. Точка выгрузки: Бескудниковский бульвар, 52к1\nРоссия, Москва\n'

In [21]:
"""
Алексей Мельхер:
Заявка на доставку 
1. Дата отгрузки: 26.04-27.04
2. Марка: ЦЕМ I 42,5Н Костюковичи 
3. Количество: 105 тонн
4. Продажа от клиента: ООО СЗТК
5. С Люберец 
6. Покупатель груза: ООО БетонПром
7. Грузополучатель(при оформлении ТТН): ООО БетонПром
9. Пункт разгрузки (факт.): 
Октябрьский улица Старый двор промзона
"""

'\nАлексей Мельхер:\nЗаявка на доставку \n1. Дата отгрузки: 26.04-27.04\n2. Марка: ЦЕМ I 42,5Н Костюковичи \n3. Количество: 105 тонн\n4. Продажа от клиента: ООО СЗТК\n5. С Люберец \n6. Покупатель груза: ООО БетонПром\n7. Грузополучатель(при оформлении ТТН): ООО БетонПром\n9. Пункт разгрузки (факт.): \nОктябрьский улица Старый двор промзона\n'

In [7]:
"""
Заявка на доставку  
1. Дата отгрузки:  29.04
2. Марка цемента: ЦЕМ I 42,5 Н Кричев/Костюковичи
3. Кол-во 140 тонн
4. Продажа от клиента: ООО Спарта    
5. Завод отгрузки: КПП, Солнечногорск, Люберцы 
6. Покупатель груза: ООО «Сибирский элемент Рента - К»    
7. Грузополучателя (при оформлении ТТН): ООО «Сибирский элемент Рента - К»    
8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    
9. Пункт разгрузки (факт): Калужская обл., Дзержинский р-н, д. Обухово 1Б    
10. Плательщик транспортировки (при оформлении ТТН):    
Контактный телефон 8 (965) 709-90-77 Игорь.
Приёмка круглосуточно
"""

'\nЗаявка на доставку  \n1. Дата отгрузки:  29.04\n2. Марка цемента: ЦЕМ I 42,5 Н Кричев/Костюковичи\n3. Кол-во 140 тонн\n4. Продажа от клиента: ООО Спарта    \n5. Завод отгрузки: КПП, Солнечногорск, Люберцы \n6. Покупатель груза: ООО «Сибирский элемент Рента - К»    \n7. Грузополучателя (при оформлении ТТН): ООО «Сибирский элемент Рента - К»    \n8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    \n9. Пункт разгрузки (факт): Калужская обл., Дзержинский р-н, д. Обухово 1Б    \n10. Плательщик транспортировки (при оформлении ТТН):    \nКонтактный телефон 8 (965) 709-90-77 Игорь.\nПриёмка круглосуточно\n'

In [8]:
"""
Юра Менеджер:
Заявка на доставку (автономка) 
1. Дата отгрузки: 28.04.24 
2. Марка: ЦЕМ I 42,5н БЦЗ Костюковичи 
Машина с пломбами и актуальными паспортами 
3. Количество: 40т.
4. Продажа от: ООО "Спарта"
5. Перевалка: СЗТК 
6. Покупатель: ООО "КАНТАН"
7. Грузополучатель: ООО "КАНТАН"
8. Время приёмки: к 9 
9. Контакт диспетчера: +79777728931 
10. Точка выгрузки: территориальное управление Соколовское
городской округ Солнечногорск, Московская область
"""

'\nЮра Менеджер:\nЗаявка на доставку (автономка) \n1. Дата отгрузки: 28.04.24 \n2. Марка: ЦЕМ I 42,5н БЦЗ Костюковичи \nМашина с пломбами и актуальными паспортами \n3. Количество: 40т.\n4. Продажа от: ООО "Спарта"\n5. Перевалка: СЗТК \n6. Покупатель: ООО "КАНТАН"\n7. Грузополучатель: ООО "КАНТАН"\n8. Время приёмки: к 9 \n9. Контакт диспетчера: +79777728931 \n10. Точка выгрузки: территориальное управление Соколовское\nгородской округ Солнечногорск, Московская область\n'

In [9]:
"""
Алексей Мельхер:
Заявка на доставку 
1.Дата отгрузки 28.04
2. Марка: ЦЕМ I 42.5Н Костюковичи 
3. Количество: 70 тонн     
4. Продажа от ООО "СЗТК" 
5. С Солнечногорска 
6. Покупатель ООО «М Бетон» 
7. Грузополучатель ООО «М-Бетон» 
8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 
9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината
10. Контакты для связи 8-925-758-01-54
"""

'\nАлексей Мельхер:\nЗаявка на доставку \n1.Дата отгрузки 28.04\n2. Марка: ЦЕМ I 42.5Н Костюковичи \n3. Количество: 70 тонн     \n4. Продажа от ООО "СЗТК" \n5. С Солнечногорска \n6. Покупатель ООО «М Бетон» \n7. Грузополучатель ООО «М-Бетон» \n8. Юр.Адрес грузополучателя: 109382, г. Москва, вн. тер. г. муниципальный округ Люблино, ул. Люблинская, д. 72, помещ. 26 \n9. Пункт разгрузки: Москва г, Очаковское ш., дом 5А, въезд с проезда Стройкомбината\n10. Контакты для связи 8-925-758-01-54\n'

In [10]:
"""
ИГО:
Заявка на доставку 
1. Дата отгрузки 28.04.2023 
2. Марка ЦЕМ I 42.5н Беларусь   
3. Количество 40 тонн  
4. От ООО Спарта 
5. Завод: Сзтк 
6. Покупатель ООО ""ТД"Цемент 
7. Грузополучатель: ООО "ТД"Цемент  
8. Голицыно
+7 910 404-06-14
РБУ
Можайское ш., 81
"""

'\nИГО:\nЗаявка на доставку \n1. Дата отгрузки 28.04.2023 \n2. Марка ЦЕМ I 42.5н Беларусь   \n3. Количество 40 тонн  \n4. От ООО Спарта \n5. Завод: Сзтк \n6. Покупатель ООО ""ТД"Цемент \n7. Грузополучатель: ООО "ТД"Цемент  \n8. Голицыно\n+7 910 404-06-14\nРБУ\nМожайское ш., 81\n'

In [12]:
"""
Заявка на самовывоз 
1. Дата отгрузки: 
28.04.2024 
Марка Цем I 42,5Н/ Кричев 
3. Количество машин/тонн: 70 тонн 
4. Перевалка Сзтк 
5.Покупатель груза: "ЗАО ПК ТЕРМОБЕТОН  
6. Продажа от ООО «СЗТК»

н223вм977 Музафяров Руслан
н095вм977 Трушев Давид
Т132ак977 Бабий Антон
Т132Ак977 Савинец Юрий
Н223ВМ977 Маркин Александр
Н095ВМ977 Шаркевич Алексей
"""

'\nЗаявка на самовывоз \n1. Дата отгрузки: \n28.04.2024 \nМарка Цем I 42,5Н/ Кричев \n3. Количество машин/тонн: 70 тонн \n4. Перевалка Сзтк \n5.Покупатель груза: "ЗАО ПК ТЕРМОБЕТОН  \n6. Продажа от ООО «СЗТК»\n\nн223вм977 Музафяров Руслан\nн095вм977 Трушев Давид\nТ132ак977 Бабий Антон\nТ132Ак977 Савинец Юрий\nН223ВМ977 Маркин Александр\nН095ВМ977 Шаркевич Алексей\n'

In [1]:
"""
Заявка на самовывоз 
1. Дата отгрузки: 
28.04.2024 
Цем I 42,5Н/ Кричев 
3. Количество машин/тонн: 70 тонн 
4. Перевалка Сзтк 
5.Покупатель груза: "ЗАО ПК ТЕРМОБЕТОН  
6. Продажа от ООО «СЗТК»

н223вм977 Музафяров Руслан
"""

'\nЗаявка на самовывоз \n1. Дата отгрузки: \n28.04.2024 \nЦем I 42,5Н/ Кричев \n3. Количество машин/тонн: 70 тонн \n4. Перевалка Сзтк \n5.Покупатель груза: "ЗАО ПК ТЕРМОБЕТОН  \n6. Продажа от ООО «СЗТК»\n\nн223вм977 Музафяров Руслан\n'

In [1]:
"""
1. Дата отгрузки:  29.04-10.05
2. Марка цемента: ЦЕМ I 42,5 Н Кричев/Костюковичи
3. Кол-во тонн:
29.04-140
30.04-160
01.05-120
02.05-160
03.05-120
04.05-160
05.05-120
06.05-160
07.05-120
08.05-160
09.05-120
10.05-160
4. Продажа от клиента: ООО Спарта    
5. Завод отгрузки: КПП, Солнечногорск, Люберцы 
6. Покупатель груза: ООО «Сибирский элемент Рента - К»    
7. Грузополучателя (при оформлении ТТН): ООО «Сибирский элемент Рента - К»    
8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    
9. Пункт разгрузки (факт): Калужская обл., Дзержинский р-н, д. Обухово 1Б    
10. Плательщик транспортировки (при оформлении ТТН):    
Контактный телефон 8 (965) 709-90-77 Игорь.
Приёмка круглосуточно
"""

'\n1. Дата отгрузки:  29.04-10.05\n2. Марка цемента: ЦЕМ I 42,5 Н Кричев/Костюковичи\n3. Кол-во тонн:\n29.04-140\n30.04-160\n01.05-120\n02.05-160\n03.05-120\n04.05-160\n05.05-120\n06.05-160\n07.05-120\n08.05-160\n09.05-120\n10.05-160\n4. Продажа от клиента: ООО Спарта    \n5. Завод отгрузки: КПП, Солнечногорск, Люберцы \n6. Покупатель груза: ООО «Сибирский элемент Рента - К»    \n7. Грузополучателя (при оформлении ТТН): ООО «Сибирский элемент Рента - К»    \n8. Адрес грузополучателя (юрид): 249850, Калужская обл., Дзержинский р-н, д. Обуховской, д 1Б.    \n9. Пункт разгрузки (факт): Калужская обл., Дзержинский р-н, д. Обухово 1Б    \n10. Плательщик транспортировки (при оформлении ТТН):    \nКонтактный телефон 8 (965) 709-90-77 Игорь.\nПриёмка круглосуточно\n'

In [2]:
"""
Сейчас получается так, что если хотя бы 1 метод не сработал, то ни один из сработавших методов не укладывает текст в соответствующие ячейки. А укладывает весь текст в одну конкретную ячейка. А как сделать так, чтоб сработавшие ячейки укладывались"""

'\nСейчас получается так, что если хотя бы 1 метод не сработал, то ни один из сработавших методов не укладывает текст в соответствующие ячейки. А укладывает весь текст в одну конкретную ячейка. А как сделать так, чтоб сработавшие ячейки укладывались'

In [1]:
x = ['мен']

In [2]:
x

['мен']

In [5]:
x = ['мен']
n = 4

# Проверяем, если n больше 1, добавляем в список x элемент 'мен' n раз
if n > 1:
    x += ['мен'] * (n - 1)

print(x)


['мен', 'мен', 'мен', 'мен']


In [24]:
man = ['мен']

In [25]:
n=2

In [26]:
man += [man][0] * (n - 1) if n > 1 else []

In [27]:
man

['мен', 'мен']

In [17]:
[man]

[['мен', 'мен']]